### import MAGEMin from julia using juliacall

In [ ]:
import juliacall
import numpy as np

MAGEMin_C = juliacall.newmodule("MAGEMin_C")
MAGEMin_C.seval("using MAGEMin_C")

from juliacall import Main as jl, convert as jlconvert

In [ ]:
import pyMAGEMin

In [ ]:
### Find the solidus and liquidus when P = 8 kbar

In [ ]:
db   = "ig"  # database: ig, igneous (Holland et al., 2018) mp, metapelite (White et al 2014b)
data = MAGEMin_C.Initialize_MAGEMin(db, verbose=False)
test = 0         #KLB1
data = MAGEMin_C.use_predefined_bulk_rock(data, test)
P    = 8.0
# out  = MAGEMin_C.point_wise_minimization(P,T, data)

#### Determine the liquid fraction of bulk rock to get solidus and liquidus


In [ ]:
liq_frac_vals = []
temp = np.linspace(1000, 2000, 1001)
for T in temp:
    out = MAGEMin_C.single_point_minimization(P, T, data)
    liq_frac = pyMAGEMin.MAGEMin_functions.phase_frac(phase="liq", MAGEMinOutput=out, sys_in='mol')
    liq_frac_vals.append( liq_frac )

In [ ]:
### find where liquidus first becomes 1 (liquidus)
liquidus_T = temp[np.where(np.array(liq_frac_vals) == 1.0)[0][0]]
### find where liquidus is 0 for last time (solidus)
solidus_T  = temp[np.where(np.array(liq_frac_vals) == 0.0)[0][-1]]

In [ ]:
import matplotlib.pyplot as plt


plt.plot(temp,  liq_frac_vals , c='k', ls=':')
plt.scatter(liquidus_T, 1, marker='x', c='red')
plt.scatter(solidus_T, 0, marker='x', c='green')

In [ ]:
print(f'solidus = {solidus_T}°C, liquidus = {liquidus_T}°C')

### Find solidus and liquidus to nearest degree

In [ ]:
initial_T = 1200.
solidus_T = float( initial_T )
out = MAGEMin_C.single_point_minimization(P, liquidus_T, data)
liq_frac = pyMAGEMin.MAGEMin_functions.phase_frac(phase="liq", MAGEMinOutput=out, sys_in='mol')

while liq_frac > 0:
    solidus_T -= 1.
    out = MAGEMin_C.single_point_minimization(P, solidus_T, data)
    liq_frac = pyMAGEMin.MAGEMin_functions.phase_frac(phase="liq", MAGEMinOutput=out, sys_in='mol')
    print(liq_frac, solidus_T)

In [ ]:
initial_T = 1800.
liquidus_T = float( initial_T )
out = MAGEMin_C.single_point_minimization(P, liquidus_T, data)
liq_frac = pyMAGEMin.MAGEMin_functions.phase_frac(phase="liq", MAGEMinOutput=out, sys_in='mol')

while liq_frac < 1:
    liquidus_T += 1.
    out = MAGEMin_C.single_point_minimization(P, liquidus_T, data)
    liq_frac = pyMAGEMin.MAGEMin_functions.phase_frac(phase="liq", MAGEMinOutput=out, sys_in='mol')
    print(liq_frac, liquidus_T)

In [ ]:
print(f'solidus = {solidus_T}°C, liquidus = {liquidus_T}°C')

### These have been wrapped into functions within the package

In [ ]:
from pyMAGEMin.functions.MAGEMin_functions import PhaseFunctions

In [ ]:
initial_solidus_T = 1200.0  
initial_liquidus_T = 1800.0 

phaseFunctions = PhaseFunctions()

In [ ]:
solidus_T = phaseFunctions.find_phase_in(P, initial_solidus_T, data, phase='liq', sys_in='mol', precision=1.)
print(f"Determined Solidus Temperature: {solidus_T:.2f} °C")

liquidus_T = phaseFunctions.find_phase_saturation(P, initial_liquidus_T, data, phase='liq', sys_in='mol', precision=1.)
print(f"Determined Liquidus Temperature: {liquidus_T:.2f} °C")

In [ ]:
print(f'solidus = {solidus_T}°C, liquidus = {liquidus_T}°C')